<a href="https://colab.research.google.com/github/anjisun221/css_codes/blob/main/ay21t1/Lab03_text_analysis/Lab3_text_analysis_national_day_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Lab 3 - Text-as-Data, National Day Speech analysis

In this lab, you will learn:
* How to clean texts
* How to count words from texts and draw wordcloud from word frequencies
* How to do comparative text analysis by extracting representative words from a corpus using lod-odds analysis

This lab is written by Jisun AN (jisunan@smu.edu.sg) and Michelle KAN (michellekan@smu.edu.sg).


In [ ]:
# Add Google Drive as an accessible path (Optional if you are running from Jupyter Notebook)
from google.colab import drive
drive.mount('/content/drive')

# change path to the designated google drive folder
# otherwise, data will be saved in /content folder which you may have issue locating
%cd /content/drive/My Drive/Colab Notebooks/

In [ ]:
mypath = "."

# 0. import utility functions

#### We've created a few utility functions. 

```read_word_count_file()``` : the input is csv file and the output is dictionary of word counts where the key is word and the value is the frequency of the words. e.g., {'singapore': 6472873} The input file must be a CSV file where each line is a pair of word and its counrt (e.g., singapore, 6472873).  


```read_word_count_file_online()``` : this helps to import the data from online file. The input is url of the csv file and the output is dictionary of word counts where the key is word and the value is the frequency of the words. e.g., {'singapore': 6472873} The input file must be a CSV file where each line is a pair of word and its counrt (e.g., singapore, 6472873).  

In [ ]:
### Let's download ```smt203util.py``` Below code should download the file in the same folder where your jupyter notebook is. 
!wget https://raw.githubusercontent.com/anjisun221/css_codes/main/ay21t1/Lab03_text_analysis/smt203util.py


In [ ]:
### Let's import all function from smt203util! 
from smt203util import *


# 1. Getting the data

In this lab, we will use national day speech data. 

This data is manually annotated by humans according to their aspect and sentiment. 

One review may have two or more aspects and thus two ore more sentiment. 

We note that we excluded those conflicting reviews.

"restaurant_reviews.tsv" is tab-separated file which fields are: 

- `sid` is review id
- `text` is a review
- `aspect` refers to the review area of interest. It consists of any of these five labels: <i>food, service, ambience, price</i> 
- `sentiment` consists of one of these labels: <i>positive, negative, neutral</i>

In [ ]:
### Import Pandas to analyze the data
import pandas as pd


In [ ]:
### Read the file using Pandas 'read_table' function (either read_table, read_csv is fine)
df = pd.read_table("https://raw.githubusercontent.com/anjisun221/css_codes/main/ay21t1/Lab03_text_analysis/national_day_speech_1996_1991_2004_2020.tsv", sep="\t")
# df = pd.read_table("./national_day_speech_1996_1991_2004_2020_extended.tsv", sep="\t")

### print the size of data frame (data frame == table)
print(df.shape)
df.head()


In [ ]:
### to see the entire text 
pd.set_option('display.max_colwidth', 150)
df.head()

In [ ]:
df.speech.unique()

In [ ]:
df['speech'].value_counts()

# 2. Cleaning the data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate. Here are a bunch of things you can do to clean your data. We're going to execute just the common cleaning steps here and the rest can be done at a later point to improve our results.

**Common data cleaning steps on all text:**
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text
* Remove stop words

**More data cleaning steps after tokenization:**
* Stemming / lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos
* And more...

## Round 1. Let's convert text to lowercase, remove punctuations, remove words containing numbers.

Python has a built-in package called `re`, which can be used to work with Regular Expressions. Regular Expression, is a sequence of characters that forms a search pattern.<br>

The `re.sub()` function can be used to replace substrings. The syntax `re.sub(pattern,repl,text)` replaces the pattern matches in text with repl. In the following code, it is used to remove punctuations and remove words containing number. You can read up about `re.sub()` [here](https://www.w3schools.com/python/python_regex.asp) and [here](https://www.pythonforbeginners.com/regex/regular-expressions-in-python).

In [ ]:
# Apply a first round of text cleaning techniques
import re 
import string

def clean_text_round1(text):
    '''Make text lowercase, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text



In [ ]:
# Let's take a look at the updated text
df['clean_text_1'] = df['text'].apply(clean_text_round1)
df.head()


## Additional exercise 1 (optional)

1. Get more familar with regular expression below. 

2. Can you remove url from a tweet using regular expression? 


In [ ]:
# Examples of Regular expression 
import re # Need module 're' for regular expression

# Try find: re.findall(regexStr, inStr) -> matchedSubstringsList
# r'...' denotes raw strings which ignore escape code, i.e., r'\n' is '\'+'n'
# [0-9] matches any digit; [A-Za-z] matches any uppercase or lowercase letters.
# + means one or more
print(re.findall(r'[0-9]+', 'abc123xyz')) # Return a list of matched substrings.  
print(re.findall(r'[0-9]+', 'abcxyz')) # Return []
print(re.findall(r'[0-9]+', 'abc00123xyz456_0')) # Return ['00123', '456', '0']


# Try substitute: re.sub(regexStr, replacementStr, inStr) -> outStr
# Below code will replace all number block to *
print(re.sub(r'[0-9]+', r'*', 'abc00123xyz456_0')) # Return 'abc*xyz*_*'

In [ ]:
# Here's an example tweet
mytweet = "New pre-print that @GruppiMauricio, @sibel_adali and I have been holding on to for a while: https://arxiv.org/abs/2101.10973. The goal was to leverage content sharing practices by news outlets in news veracity detection. Thread."
mytweet


Write the code to remove URL from the above tweet.

As a result, you should see 

'New pre-print that @GruppiMauricio, @sibel_adali and I have been holding on to for a while:  The goal was to leverage content sharing practices by news outlets in news veracity detection. Thread.'

In [ ]:
text = # Write your code 
text


## Round 2. Let's remove stopwords. 

A stop word is a commonly used word (such as "the", "a", "an", "in"). For some analysis, like looking into top words, those stop words are often meaningless, and thus we remove them.

The [Natural Language Toolkit (nltk)](https://www.nltk.org/api/nltk.html) is a Python package for natural language processing. We will import the library for the removal of stop words.

In [ ]:
import nltk
nltk.download('stopwords')

The `nltk` library has a list of stopwords stored in 16 different languages. We will retrieve the list of English stop words using `stopwords.words('english')`

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
def clean_text_round2(text):
    return ' '.join([word for word in text.split() if word not in (stop)])


The expression `[word for word in x.split() if word not in (stop)]` is a list comprehension.<br>
List comprehension offers a shorter syntax when you want to create a new list based on the values of an existing list.<br> Syntax: `new_list = [expression for item in iterable list if condition == True]`<br> 
You can read up about list comprehension [here](https://www.w3schools.com/python/python_lists_comprehension.asp)

In [ ]:
df['clean_text_2'] = df['clean_text_1'].apply(clean_text_round2)
df.head()

**NOTE:** This data cleaning aka text pre-processing step could go on for a while, but we are going to stop for now. After going through some analysis techniques, if you see that the results don't make sense or could be improved, you can come back and make more edits such as:
* Mark 'outstanding' and 'outstand' as the same word (stemming / lemmatization)
* Combine 'thank you' into one term (bi-grams)
* And a lot more...

# 3. Unigram analysis - Counting words (and save it to a file) 



In [ ]:
## Create ```word_counts``` folder 
import os
os.makedirs('word_counts', exist_ok=True)

In [ ]:
# this function create a dictionary with word counts from dataframe 
def count_words_from_dataframe(df):
    result_dict = {}
    # iterate rows of dataframe 
    for index, row in df.iterrows():
        text = row['clean_text_2']
        
        # this will split a sentence into words 
        tokens = text.split()
        
        # iterate each word 
        for i in range(0, len(tokens)):
            token = tokens[i]
            try:
                result_dict[token] += 1
            except KeyError:
                result_dict[token] = 1
                    
    return result_dict


In [ ]:
list_of_speeches = list(df.speech.unique())
list_of_speeches

In [ ]:
import csv

count_type = 'unigram'

for each_speech in list_of_speeches:
    print(each_speech)
    
    ## query helps to filter rows of dataframe given a condition 
    df_speech = df.query('speech == @each_speech')
    print(df_speech.shape)
    
    ## this function will return a dictionary of words and frequency
    result = count_words_from_dataframe(df_speech)

    ## soring the words based on their frequency
    sorted_dic = ((k, result[k]) for k in sorted(result, key=result.get, reverse=True))
    
    ## write the dictionary in a file
    with open(f"./word_counts/{each_speech}_{count_type}_counts.csv", 'w') as fp:
        writer = csv.writer(fp, delimiter=',')
        writer.writerows(sorted_dic)


## Let's draw WordCloud using unigrams!

In [ ]:
## This it OPTIONAL if you are running the current notebook using Google Colab
!conda install --yes -c conda-forge wordcloud

In [ ]:
### Import relevant libraries
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import numpy as np
import re
from PIL import Image
from os import path
import csv

### this function will creat wordcloud based on word frequencies and save them into files under plot. 
def makeImage(termDict, outputfile):
    
    wc = WordCloud(max_font_size=60, width=1280, height=720, background_color="white")    

    ### generate word cloud using frequencies!
    wc.generate_from_frequencies(termDict)
    wc.to_file(outputfile+".png")
    wc.to_file(outputfile+".pdf")
    
    ### show the figure
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.show() 
    

In [ ]:
### Create ```plot``` folder 
os.makedirs('plot', exist_ok=True)

In [ ]:
speech_name = "2004_lee_hsien_loong"
### We're using one of the utility function! "read_word_count_file"
fullTermsDict = read_word_count_file(f'{mypath}/word_counts/{speech_name}_unigram_counts.csv')
outputfile = f"./plot/wordcloud_count_unigram_{speech_name}"
makeImage(fullTermsDict, outputfile)


In [ ]:
speech_name = "2020_lee_hsien_loong"
### We're using one of the utility function! "read_word_count_file"
fullTermsDict = read_word_count_file(f'{mypath}/word_counts/{speech_name}_unigram_counts.csv')
outputfile = f"./plot/wordcloud_count_unigram_{speech_name}"
makeImage(fullTermsDict, outputfile)


### Exercise 1. Count bigram 

You task is to write a function named ```count_bigrams_from_dataframe``` whose input is dataframe, ```df``` and output is the dictionary of bigrams with their counts. 

Then, can you also save them into file after sorting the bigrams based on their counts? 

(Optional) You may draw word clouds with bigrams as well! 


In [ ]:
def count_bigrams_from_dataframe(df):
    
    result_dict = {}                    
    for index, row in df_speech.iterrows():
        text = row['clean_text_2']
        tokens = text.split()
        
        ## write your code 
        
    return result_dict



In [ ]:
count_type = 'bigram'

for each_speech in list_of_speeches:
    print(each_speech)
    df_speech = df.query('speech == @each_speech')
    print(df_speech.shape)
    
    ## write your code 


# 4. Find representative words by using log odds 

### In this example, we will compare 2004 and 2020 National Day speeches by PM Lee Hsien Loong. 

We will detect unigrams that are over-represented in either 2004 or 2020 speech. 

To do so, we use log-odds ratios with informative Dirichlet priors (Monroe, Colaresi, and Quinn 2008). This method estimates the log-odds ratio of each word between two corpora i and j given the prior frequencies obtained from a back ground corpus. 

This method identifies representative words of a corpus i in comparison with another corpus j and the common corpus (global/background corpus). 

#### We've created a few functions for computing log odds values and finding the representativ words.

```calculate_log_odds_idp(global_counts, counts_i_name, counts_i_dict, counts_j_name, counts_j_dict)``` : it returns the log odds value (```log_odds_z_score```) for each word, which indicates the significance of the word in the corresponding corpus. You should provide background corpus (we call ```global_counts```), two names of the corpora: ```counts_i_name``` and ```counts_j_name``` (for naming the file), and two dictionaries of word counts from each corpus: ```counts_i_dict``` and ```counts_j_dict```. 

```find_discriminative_words(top_words_df, threshold_i, threshold_j, num_i, num_j, mypath)```: This function will filter out those words that appear less than k times in each corpus (`threshold_i` and ```threshold_j```), rank the words based on lod-odds z-scores and select top-N representative words (```num_i```, ```num_j```), and generate a csv file where each row contains (word, log_odds_z_score) for each of the two corpora .  

For details about how to compute log odds z-score, see: "Monroe, B. L.; Colaresi, M. P.; and Quinn, K. M. 2008. Fightin’words: Lexical feature selection and evaluation for identifying the content of political conflict. Political Analysis 16(4):372–403."

In [ ]:
### Getting global word counts (frequency of unigram in Google News datasets)
target_url = "https://raw.githubusercontent.com/anjisun221/css_codes/main/ay21t1/Lab03_text_analysis/1gram_englishall_count.csv"
global_counts = read_word_count_file_online(target_url)
print("The number of unigrams=", len(global_counts))


In [ ]:
### Removing stopwords from the word list and 
stopwords_list = get_stopwords()
global_counts = {k: v for k, v in global_counts.items() if (k not in stopwords_list) and (len(k) > 2)}
print("After excluding stop words: ", len(global_counts))


#### Let's load word frequency file and create dictionary of word counts for the two speeches!

In [ ]:
# counts_i_name = "1966_lee_kuan_yew"
# counts_i_name = "1991_goh_chok_tong"
counts_i_name = "2004_lee_hsien_loong"
# counts_j_name = "2020_lee_hsien_loong"
counts_i = read_word_count_file(f"{mypath}/word_counts/{counts_i_name}_unigram_counts.csv")

### we filter out words that exist in our background corpus
counts_i_dict = {k: v for k, v in counts_i.items() if k in global_counts}


In [ ]:
# counts_j_name = "1966_lee_kuan_yew"
# counts_j_name = "1991_goh_chok_tong"
# counts_j_name = "2004_lee_hsien_loong"
counts_j_name = "2020_lee_hsien_loong"
counts_j = read_word_count_file(f"{mypath}/word_counts/{counts_j_name}_unigram_counts.csv")

### we filter out words that exist in our background corpus
counts_j_dict = {k: v for k, v in counts_j.items() if k in global_counts}


In [ ]:
### this function will return log-odds values. 
top_words_df = calculate_log_odds_idp(global_counts, counts_i_name, counts_i_dict, counts_j_name, counts_j_dict)


### Interpretation lad z-score (log_odds_z_score) 

Consider it as a value that indicate the significance of the word in a corpus. Given that we have two corpora, i and j, the order matters when interpreting this z-score. For corpus i, the lower the z-score, more significant the word is while for corpus j, the higher the z-score, more significant the word is.

The word frequency is also important, if the word frequency is low, those words may not be meaningful. So, we filter words based on the two condition: word frequency and z-score. 

In the below example, 
for corpus i, we will consider words that have lower z-scores and that appear at least twice in the corpus. 

For corpus j, we will consider words that have higher z-scores and that appear at least twice in the corpus. 

Note that, we choose 2 as a threshold for word frequency, but usually it should be much higher if we have larger corpus. Our speech data is small, so we have to choose 2. Normally, we would use 50 or 100 as a minimum number of word frequency. 

In [ ]:
### Below shows the representative words of 2004 National Day speech by PM Lee. 
top_words_df[top_words_df[counts_i_name] >= 2].iloc[::-1].head(20)

In [ ]:
### Below shows the representative words of 2020 National Day speech by PM Lee. 
top_words_df[top_words_df[counts_j_name] >= 2].head(20)


In [ ]:
### Below function will simply help you to print the above table into file, so that we can use it for drawing word cloud. 
### The output of this function is a csv file where each row contains (word, log_odds_z_score) for the two corpora. 
### threshold_i and threshold_j are the threshold of word frequency. 
### num_i and num_j are the number of representative words it will write in file. 
find_discriminative_words(top_words_df, threshold_i=2, threshold_j=2, num_i=20, num_j=20, mypath='.')


Note that the above function (`find_discriminative_words()`) results in creating two files where each row is a word and its log-odds value and each file is for each speech!!  

## Let's draw WordClouds based on log odds values 

### Exercise 2. Draw WordClouds based on the log odds values for each speech and write the WordClouds in a file. 

You can use the resulting files of `find_discriminative_words()` to draw the wordcloud. 

Hint 1: You can simply consider "log-odds value" as a frequency. 
Hint 2: Note that you need to create one wordcloud for each speech. 

In [ ]:
### Draw a WordCloud for 2004 National Day speech by PM Lee.

[Write your code] 


In [ ]:
# Draw a WordCloud for 2020 National Day speech by PM Lee. 

[Write your code] 